In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Conv1D
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent, CEMAgent, DDPGAgent, SARSAAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import numpy as np

from gym import Env
from gym.spaces import Box, Discrete

from os.path import exists

import matplotlib.pyplot as plt
import pandas as pd

rng = np.random.default_rng()

In [2]:
class TakeItEasyEnv(Env):
    def __init__(self):
        self.action_space = Discrete(20, start=1)
        self.observation_space = Box(low=-100, high=96100, shape=(1,1,20,3), dtype=np.int32)
        self.state = np.zeros(shape=(20, 3), dtype=np.int32)
        self.episode_length = 19
        self.round_no = 0
        self.pieces = np.array([
            [2,1,3], [2,1,4], [2,1,8], [6,1,3], [6,1,4], [6,1,8], [7,1,3], [7,1,4], [7,1,8], 
            [2,5,3], [2,5,4], [2,5,8], [6,5,3], [6,5,4], [6,5,8], [7,5,3], [7,5,4], [7,5,8],
            [2,9,3], [2,9,4], [2,9,8], [6,9,3], [6,9,4], [6,9,8], [7,9,3], [7,9,4], [7,9,8]])
        self.cases = np.array([
            [[1,4,8,0,0], [2,5,9,13,0], [3,6,10,14,17], [7,11,15,18,0], [12,16,19,0,0]],
            [[1,2,3,0,0], [4,5,6,7,0], [8,9,10,11,12], [13,14,15,16,0], [17,18,19,0,0]], 
            [[3,7,12,0,0], [2,6,11,16,0], [1,5,10,15,19], [4,9,14,18,0], [8,13,17,0,0]]       
            ], np.int32)
        self.last_reward = 0
        self.selected_pieces = rng.choice(self.pieces, size=self.episode_length, replace=False)


        self.state[0] = self.selected_pieces[0]

    def count_points(self, state):
        points=0
        for i in range(3):
            for j in range(5):
                first_value = state[self.cases[i][j][0],i]
                for k in range(5):
                    if(self.cases[i][j][k] == 0):
                        points += first_value*k
                        break
                    if(self.state[self.cases[i][j][k], i] != first_value):
                        break
                    elif(k == 4):
                        points += first_value*(k+1)
                        break
        return points

    def reset(self):
        self.round_no = 0
        self.last_reward = 0
        self.state=np.zeros(shape=(20, 3), dtype=np.int32)
        self.selected_pieces = rng.choice(self.pieces, size=self.episode_length, replace=False)
        self.state[0] = self.selected_pieces[0]
        return self.state
    
    def step(self, action):
        self.round_no += 1
        reward = 0

        if(np.all(self.state[action] != 0) and self.round_no <= self.episode_length):
            reward = -100
            done = True
        else:
            if(self.round_no == self.episode_length):
                self.state[action] = self.state[0]
                reward = 2*self.count_points(self.state) - self.last_reward
                done = True
            else:
                self.state[action] = self.state[0]
                self.state[0] = self.selected_pieces[self.round_no]
                reward = self.count_points(self.state) - self.last_reward
                self.last_reward = self.count_points(self.state)
                done = False

        info = {}

        return self.state, reward, done, info


In [3]:
env=TakeItEasyEnv()

states = env.observation_space.shape
actions = env.action_space.n


In [4]:
def build_model(states, actions):
    model = Sequential()
    model.add(InputLayer(input_shape=(1,20,3)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 20, 256)        1024      
                                                                 
 dense_1 (Dense)             (None, 1, 20, 512)        131584    
                                                                 
 dense_2 (Dense)             (None, 1, 20, 256)        131328    
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                                                 
 dense_3 (Dense)             (None, 20)                102420    
                                                                 
Total params: 366,356
Trainable params: 366,356
Non-trainable params: 0
_________________________________________________________________


In [39]:
def build_agent(model, actions):
    memory = SequentialMemory(limit=200000000, window_length=1)
    policy = BoltzmannQPolicy()
    dqn = DQNAgent(model=model, nb_actions=actions, memory=memory, nb_steps_warmup=100, target_model_update=1e-2, policy=policy)
    
    return dqn

dqn = build_agent(model, actions)

dqn.compile(Adam(learning_rate=0.5e-3), metrics=['mae'])

In [8]:
def save_highscore(highscore):
    with open('highscore.txt', 'w') as f:
        f.write(str(highscore))

def load_highscore():
    try:
        with open('highscore.txt', 'r') as f:
            highscore = int(f.read())
    except:
        highscore = -100
    return highscore

steps = 500000
highest_reward = load_highscore()
for i in range(int(steps)//10000):
    print('Step: {}'.format(i*10000))
    try:
        dqn.load_weights('dqn_take-it-easy_weights.hdf5')
    except:
        print('Could not load weights')
    
    train_results = dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

    results_reward = np.mean(train_results.history['episode_reward'])
    print('Highest reward: {}'.format(highest_reward))
    print('Train reward: {}'.format(results_reward))

    if(results_reward > highest_reward):
        print('Model improved')
        dqn.save_weights('dqn_take-it-easy_weights.hdf5', overwrite=True)
        highest_reward = results_reward
        save_highscore(highest_reward)
    else:
        print('Model did not improve')  

Step: 0
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 248s 25ms/step - reward: 8.0364
done, took 248.380 seconds
Highest reward: 158.772
Test reward: 162.92
Model improved
Step: 10000
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 252s 25ms/step - reward: 8.4760
done, took 252.017 seconds
Highest reward: 162.92
Test reward: 171.46
Model improved
Step: 20000
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 258s 26ms/step - reward: 8.72460s - rew
done, took 258.378 seconds
Highest reward: 171.46
Test reward: 174.7
Model improved
Step: 30000
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 268s 27ms/step - reward: 8.9404
done, took 267.871 seconds
Highest reward: 174.7
Test reward: 174.71
Model improved
Step: 40000
Training for 10000 steps ...
Interval 1 (

In [41]:
results = dqn.test(env, nb_episodes=150, visualize=False)
print(np.max(results.history['episode_reward']))

Testing for 150 episodes ...
Episode 1: reward: 142.000, steps: 19
Episode 2: reward: 148.000, steps: 19
Episode 3: reward: 196.000, steps: 19
Episode 4: reward: 94.000, steps: 19
Episode 5: reward: 142.000, steps: 19
Episode 6: reward: 142.000, steps: 19
Episode 7: reward: 196.000, steps: 19
Episode 8: reward: 178.000, steps: 19
Episode 9: reward: 62.000, steps: 19
Episode 10: reward: 196.000, steps: 19
Episode 11: reward: 190.000, steps: 19
Episode 12: reward: 62.000, steps: 19
Episode 13: reward: 196.000, steps: 19
Episode 14: reward: 156.000, steps: 19
Episode 15: reward: 196.000, steps: 19
Episode 16: reward: 94.000, steps: 19
Episode 17: reward: 136.000, steps: 19
Episode 18: reward: 196.000, steps: 19
Episode 19: reward: 256.000, steps: 19
Episode 20: reward: 196.000, steps: 19
Episode 21: reward: 100.000, steps: 19
Episode 22: reward: 142.000, steps: 19
Episode 23: reward: 218.000, steps: 19
Episode 24: reward: 196.000, steps: 19
Episode 25: reward: 156.000, steps: 19
Episode 2

In [ ]:
cases = np.array([
    [[1,4,8,0,0], [2,5,9,13,0], [3,6,10,14,17], [7,11,15,18,0], [12,16,19,0,0]],
    [[1,2,3,0,0], [4,5,6,7,0], [8,9,10,11,12], [13,14,15,16,0], [17,18,19,0,0]], 
    [[3,7,12,0,0], [2,6,11,16,0], [1,5,10,15,19], [4,9,14,18,0], [8,13,17,0,0]]       
    ], np.int32)


#state = np.full(shape=(19,3), fill_value=1, dtype=np.int32)
state = np.array([
            [0,9,0], [0,9,0], [0,0,0],
        [0,0,0], [0,0,0], [0,0,0], [0,0,0],
    [0,9,0], [0,9,0], [0,9,0], [0,9,0], [0,0,0],
        [0,0,0], [0,9,0], [0,9,0], [0,9,0],
            [0,0,0], [0,0,0], [0,0,0]
    ])

#state = rng.choice(pieces,19, replace=False)

visualize = f'''
                ____
               /    \                               
          ____/      \____                                    
              {state[7]}
         /    \      /    \                                         
    ____/      \____/      \____                                 
        {state[3]}     {state[12]}                                      
   /    \      /    \      /    \                                       
  /      \____/      \____/      \                                  
  {state[0]}     {state[8]}     {state[16]}                                       
  \      /    \      /    \      /                                                      
   \____/      \____/      \____/                       
        {state[4]}     {state[13]}                                 
   /    \      /    \      /    \                                       
  /      \____/      \____/      \                                      
  {state[1]}     {state[9]}     {state[17]}                                           
  \      /    \      /    \      /                                      
   \____/      \____/      \____/                                       
        {state[5]}     {state[14]}                                             
   /    \      /    \      /    \                                       
  /      \____/      \____/      \                                      
  {state[2]}     {state[10]}    {state[18]}                                           
  \      /    \      /    \      /                                                  
   \____/      \____/      \____/
        {state[6]}     {state[15]}                                                                 
        \      /    \      /                                                        
         \____/      \____/                                                     
              {state[11]}                                                   
              \      /                                                      
               \____/                                                   
''' 

#print(state)

points=0
for i in range(3):
    for j in range(5):
        first_value = state[cases[i][j][0]-1,i]
        for k in range(5):
            if(cases[i][j][k] == 0):
                points += first_value*k
                break
            if(state[cases[i][j][k]-1, i] != first_value):
                break
            elif(k == 4):
                points += first_value*(k+1)
                break

print(points)
print(visualize)

                